# Errors and Exceptions in Lambda Expressions

# Exceptions

In Python, `Exceptions` are the main way to deal with errors. Whenever an error occurs, an `Exception` object is 'raised'. This causes the code to abort, until the Exception is caught in a `try: … except: …` statement.

In [2]:
def add_str(s):
    
    try:
        return sum([int(i) for i in s.split('+')])
    except AttributeError:
        return None

print(add_str(1+2))

None


In [3]:
print(add_str('1+2'))

3


#### The curse of statements


But `try` is a *statement*, and you can therefore not use it in `lambda` expressions! Unlike for most other statements, Python does not offer an expression alternative to the `try` statement.

In [4]:
l_add_str = lambda s: sum([int(i) for i in s.split('+')])

# we cannot catch error in lambda function (we cannot use try-except in lambda function)
print(l_add_str(1+2))

AttributeError: 'int' object has no attribute 'split'

In [5]:
print(l_add_str('1+2'))

3


# Handling Errors

- Let's reconsider our lambda

In [6]:
l_add_str = lambda s: sum([int(i) for i in s.split('+')])

#### A Maybe-like decorator

The `Maybe monad` is not very Pythonic. But we can do something similar using a decorator.

In [15]:
def maybe(fnc):
    
    def inner(*args):
        
        for a in args:
            if isinstance(a, Exception):
                return a
        try:
            return fnc(*args)
        except Exception as e:
            return e
        
    return inner

safe_add_str = maybe(lambda s: sum([int(i) for i in s.split('+')]))
print(safe_add_str(1+2))

'int' object has no attribute 'split'


In [16]:
print(safe_add_str('1+2'))

3


# A fully functional, interactive calculator

#### Our functional calculator … so far

This is the calculator that we implemented in the first section. But it suffers from a few drawbacks:

- No input validation
- No looping

In [17]:
OPERATORS = '+', '-', '*', '/'


def f_get_number():
    return int(input('Enter an integer: '))


def f_get_operator():
    return input('Enter an operator (+, -, *, /): ')


def f_calculate(number1, operator, number2):
    return number1+number2 if operator == '+' \
        else number1-number2 if operator == '-' \
        else number1/number2 if operator == '/' \
        else number1*number2 if operator == '*' \
        else None    


def f_main():
    return f_calculate(
        f_get_number(),
        f_get_operator(),
        f_get_number(),
        )


print('The result is: %s' % f_main())

The result is: 3


In [18]:
f_calculate(1, '+', 2)

3

#### Let's get to work!

Our toolkit contains:

- Lambda expressions
- Decorators
- Higher-order functions

In [19]:
OPERATORS = '+', '-', '*', '/'


def maybe(fnc):
    
    """Turns Exceptions into return values."""
    
    def inner(*args):
        
        for a in args:
            if isinstance(a, Exception):
                return a
        try:
            return fnc(*args)
        except Exception as e:
            return e
        
    return inner


def repeat(fnc, until):
    
    """Repeats a function until its return value meets
    the stop criterion."""
    
    def inner(*args):

        while True:
            result = fnc(*args)
            if until(result):
                return result
            
    return inner


is_int = lambda i: isinstance(i, int)
get_number = lambda: int(input('Enter an integer: '))
safe_get_number = repeat(maybe(get_number), until=is_int)

is_operator = lambda o: o in OPERATORS
get_operator = lambda: input('Enter an operator (+, -, *, /): ')
safe_get_operator = repeat(get_operator, until=is_operator)


calculate = lambda number1, operator, number2: \
    number1+number2 if operator == '+' \
    else number1-number2 if operator == '-' \
    else number1/number2 if operator == '/' \
    else number1*number2 if operator == '*' \
    else None    

main = lambda: calculate(
        safe_get_number(),
        safe_get_operator(),
        safe_get_number(),
        )

forever = lambda retval: False
main_loop = repeat(lambda: print(main()), until=forever)

main_loop()

3
5
4


KeyboardInterrupt: Interrupted by user